In [10]:
from dotenv import load_dotenv
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai.tools import tool
from IPython.display import display, Markdown
import requests

load_dotenv()

True

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [3]:
synth_llm_small = LLM(
    model="anthropic/claude-3-5-haiku-20241022",
    temperature=0.2
)
synth_llm_large = LLM(
    model="anthropic/claude-3-5-sonnet-20241022",
    temperature=0.7
)
search_llm = LLM(model = "groq/llama-3.3-70b-versatile")

In [4]:
@tool
def searxng_search(query: str) -> str:
    """
    Search the web using a locally hosted SearxNG instance.
    Args:
        query (str): Search term
    Returns:
        str: Top results
    """
    SEARXNG_URL = "http://localhost:8888/search"
    params = {'q': query, 'format': 'json'}
    response = requests.get(SEARXNG_URL, params=params)

    if response.status_code == 200:
        results = response.json()
        return "\n".join(
            f"{res['title']} - {res['url']}" for res in results['results']
        )
    else:
        return f"Error: {response.status_code}"

In [5]:
finhealth_sales_analyst_agent = Agent(
    config=agents_config['finhealth_sales_analyst'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=synth_llm_small,
    max_rpm=25,
    respect_context_window=True
)
chief_sales_analyst_agent = Agent(
    config=agents_config['chief_sales_analyst'],
    llm=synth_llm_large,
    max_rpm=25,
    respect_context_window=True
)

In [6]:
finhealth_research_task = Task(
    config=tasks_config['finhealth_research_task'],
    agent=finhealth_sales_analyst_agent
)
finhealth_research_task_1 = Task(
    config=tasks_config['finhealth_research_task_1'],
    agent=finhealth_sales_analyst_agent
)
finhealth_research_task_6 = Task(
    config=tasks_config['finhealth_research_task_6'],
    agent=finhealth_sales_analyst_agent
)
finhealth_report_task = Task(
   config=tasks_config['finhealth_report_task'],
    agent=chief_sales_analyst_agent,
    context=[finhealth_research_task_1, finhealth_research_task_6]
)


finhealth_crew = Crew(
    agents=[finhealth_sales_analyst_agent],
    tasks=[finhealth_research_task],
    verbose=True
)
result = finhealth_crew.kickoff(inputs={
  'Company': 'Golden 1 Credit Union'
})
from IPython.display import display, Markdown
display(Markdown(str(result)))

In [7]:
finhealth_crew_1 = Crew(
    agents=[finhealth_sales_analyst_agent],
    tasks=[finhealth_research_task_1],
    verbose=True
)

In [8]:
result_1 = finhealth_crew_1.kickoff(inputs={
  'Company': 'Golden 1 Credit Union'
})

# Agent: Lead Sales Analyst - Financial Health and Wellness
## Task: Conduct a thorough research about Golden 1 Credit Union to evaluate their board commitment to financial health and wellness. Do a deep search on the company website, including its about page, annual reports, press releases, board statements and other information on its website showing the committment of their board to financial health and wellness. Prioritize information in the \ last 2-3 years, given that the current year is 2025. Only use information that  you found in your search.  Focus your evaluation of Golden 1 Credit Union on the following observable criteria: - Published strategic plans or annual reports mentioning financial health initiatives - Public statements from board members about financial health priorities - Dedicated financial health sections on website/marketing materials. Do not make up any information that you do not find in your search.Do not be swayed by the company's reputation, your personal 

In [9]:
display(Markdown(str(result_1)))

Golden 1 Credit Union - Board Commitment to Financial Health and Wellness Report

Overall Rating: 4/5 (Strong Commitment)

Detailed Evaluation:

1. Published Strategic Plans and Initiatives:
- In May 2023, Golden 1 launched a significant financial wellness initiative by partnering with iGrad to offer the Enrich™ personalized financial wellness platform
- Key platform features include:
  * Your Money Personality™ financial behavior assessment
  * Personalized financial education courses
  * Retirement and home affordability analyzers
  * Personal finance calculators and real-time budget tools

2. Public Statements from Leadership:
- Barbara Quan, Financial Education Manager, stated: "Californians' financial well-being is Golden 1's highest priority... [We aim to provide] foundational building blocks for financial success."
- The credit union explicitly commits to "enhancing the financial well-being of Californians and their diverse communities to create a more equitable and financially inclusive California"

3. Dedicated Financial Health Sections:
- Comprehensive Financial Wellness Center on their website
- Launched in 2018, continually updated with new resources
- Offers free financial resources to:
  * Learn healthy money management skills
  * Reduce financial stress
  * Improve credit
  * Provide peace of mind

4. Quantifiable Impact of Financial Wellness Initiatives:
Based on Enrich platform data (2020-2021), users experienced:
- 35% increase in users on track with savings goals
- 14% increase in retirement savings contributions
- 59% increase in emergency savings
- 28% increase in full monthly credit card debt payoff
- 32% reduction in financial stress

5. Institutional Commitment:
- Nearly $20 billion in assets
- Over 1 million members
- Explicitly positions financial wellness as a core mission

Supporting Evidence Links:
- Financial Wellness Center: https://www.golden1.com/financial-wellness
- iGrad Partnership Press Release: https://www.golden1.com/discoverg1/press-releases/2023/G1-CU-Teams-Up-with-iGrad%20to-Offer%20the-Enrich-Personalized-Financial-Wellness-Program-to-its-Members

Reasoning for 4/5 Rating:
- Demonstrates a robust, multi-faceted approach to financial wellness
- Quantifiable impact and continuous innovation
- Clear leadership commitment
- Comprehensive resources across multiple platforms

Deduction of 1 point due to:
- Could provide more detailed annual reporting on financial wellness outcomes
- Room for expanding personalized financial coaching

Conclusion: Golden 1 Credit Union shows a strong, data-driven commitment to member financial health and wellness, with innovative tools and a clear strategic focus on improving financial well-being.

In [7]:
finhealth_crew_6 = Crew(
    agents=[finhealth_sales_analyst_agent],
    tasks=[finhealth_research_task_6],
    verbose=True
)

In [8]:
result_6 = finhealth_crew_6.kickoff(inputs={
  'Company': 'Golden 1 Credit Union'
})

# Agent: Lead Sales Analyst - Financial Health and Wellness
## Task: Conduct a thorough research about Golden 1 Credit Union to evaluate their industry leadership.  Do a deep search on the company website, including its about page, annual reports, press releases, board statements and other information on its website showing industry leadership.  Prioritize information in the last 2-3 years, given that the current year is 2025.  Focus your evaluation of Golden 1 Credit Union on the following observable criteria: - Industry awards and recognition - Speaking slots at major conferences - Published thought leadership content - Media coverage and press mentions - Industry association leadership roles Do not make up any information that you do not find in your search.Do not be swayed by the company's reputation, your personal opinions, or documents that are purely for marketing and advertising.Be ruthlessly thorough in considering both the quality and quantity of evidence.



# Agent: Lead Sa

In [11]:
display(Markdown(str(result_6)))

Golden 1 Credit Union - Industry Leadership Evaluation Report

Leadership Rating: 4/5

Detailed Evaluation of Industry Leadership:

1. Industry Awards and Recognition:
- Fast Company's 2023 Brands That Matter List
- Newsweek's America's Greatest Workplaces for Women (2024)
- Forbes Best-in-State Employers in California (3 consecutive years)
- Top 100 Women Leaders in Finance (CEO Donna Bland recognized)
- Multiple national credit union employer accolades

Sources:
- https://www.golden1.com/blog/golden-1-honored-at-fast-companys-third-annual-list-of-brands-that-matter
- https://www.golden1.com/discoverg1/press-releases/2024/Golden%201%20Honored%20as%20one%20of%20Americas%20Greatest%20Workplaces

2. Speaking Slots and Thought Leadership:
- Podcast featuring employee experiences in Diversity, Equity, and Inclusion (DEI)
- Leadership team actively participating in industry discussions
- Sharing insights on financial wellness and community impact

Sources:
- https://www.cumanagement.com/podcasts/2023/12/golden-1-cu-takes-belief-dei-award-winning-action

3. Published Thought Leadership Content:
- Blog posts on innovation, community commitment
- 90-year anniversary reflection on industry evolution
- Detailed press releases about community initiatives
- Transparent reporting on financial performance and community impact

Sources:
- https://www.golden1.com/blog/golden-1-celebrates-90-years-of-innovation-and-community-commitment-in-california

4. Media Coverage and Press Mentions:
- Consistent positive coverage in financial and local media
- Recognition for community grants and nonprofit support
- Highlighted for innovative financial wellness programs
- Awarded $1 Million to 42 local organizations in 2025

Sources:
- https://www.golden1.com/discoverg1/press-releases/2025/Golden%201%20Awards%20$1%20Million%20to%2042%20Local%20Organizations

5. Industry Association Leadership Roles:
- Active participation in credit union industry networks
- Leadership in promoting financial inclusion
- Recognized for employee engagement and workplace culture
- Strong representation of women in leadership positions

Organizational Highlights:
- Assets: Approximately $19 billion
- Membership: Over 1.1 million members
- Employees: Over 2,000
- Serving California since 1933

Reasoning for 4/5 Rating:
Golden 1 Credit Union demonstrates strong industry leadership through consistent recognition, innovative community programs, thought leadership, and a commitment to financial wellness. The breadth and depth of their achievements, coupled with multiple national awards, position them as a top-tier credit union. The slight deduction from a perfect 5 is due to room for potential expansion in speaking engagements and broader industry association roles.

Key Strengths:
- Consistent national recognition
- Strong commitment to community and DEI
- Innovative financial wellness approach
- Transparent leadership and reporting

Recommendations for Further Leadership Enhancement:
- Increase international speaking engagements
- Expand thought leadership publications
- Continue developing industry-leading DEI initiatives